## Урок 3. 

### Практическое задание

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

In [1]:
import numpy as np
import pandas as pd

from scipy.sparse import hstack

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb, lightgbm as lgbm, catboost as catb
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix, log_loss
from sklearn.preprocessing import StandardScaler

import seaborn as sns
import itertools
import matplotlib.pyplot as plt

В рамках конкурса вам нужно предсказать наличие сердечно-сосудистых заболеваний по результатам классического врачебного осмотра. Датасет сформирован из 100.000 реальных клинических анализов, и в нём используются признаки, которые можно разбить на 3 группы:

 

Объективные признаки:

 - Возраст
 - Рост
 - Вес
 - Пол
 

Результаты измерения:

 - Артериальное давление верхнее и нижнее
 - Холестерин
 - Глюкоза
 

Субъективные признаки:

 - Курение
 - Употребление Алкоголя
 - Физическая активность
 

Возраст дан в днях. Значения показателей холестерина и глюкозы представлены одним из трех классов: норма, выше нормы, значительно выше нормы. Значения субъективных признаков — бинарны.

Все показатели даны на момент осмотра.

Таргет - наличие сердечно-сосудистых заболеваний (ССЗ)

In [2]:
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [3]:
cat_cols = ['gender', 'cholesterol', 'gluc']
for colname in cat_cols:
    df[colname] = pd.Categorical(df[colname])

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   id           70000 non-null  int64   
 1   age          70000 non-null  int64   
 2   gender       70000 non-null  category
 3   height       70000 non-null  int64   
 4   weight       70000 non-null  float64 
 5   ap_hi        70000 non-null  int64   
 6   ap_lo        70000 non-null  int64   
 7   cholesterol  70000 non-null  category
 8   gluc         70000 non-null  category
 9   smoke        70000 non-null  int64   
 10  alco         70000 non-null  int64   
 11  active       70000 non-null  int64   
 12  cardio       70000 non-null  int64   
dtypes: category(3), float64(1), int64(9)
memory usage: 5.5 MB


Разделим наши данные на тренировочную и тестовую выборки

In [5]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

К полям:
- gender, cholesterol, gluc применим OHE-кодирование
- age, height, weight, ap_hi, ap_lo - standardScaler
- smoke, alco, active - оставим пока как есть

In [6]:
models_results = {
    'models': [],
    'precision': [],
    'recall': [],
    'fscore': [],
    'roc_auc': [],
    'log_loss': []
}

In [7]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol', 'gluc']
base_cols = ['smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

Теперь объединим все наши трансформеры с помощью FeatureUnion

In [8]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

Добавим классификатор и запустим кросс-валидацию

In [9]:
classifier_LR = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])

#запустим кросс-валидацию
cv_scores = cross_val_score(classifier_LR, X_train, y_train, cv=16, scoring='neg_log_loss')

#обучим пайплайн на всем тренировочном датасете
classifier_LR.fit(X_train, y_train)
y_score_LR = classifier_LR.predict_proba(X_test)[:, 1]

In [10]:
classifier_CB = Pipeline([
    ('features',feats),
    ('classifier', catb.CatBoostClassifier(silent=True, random_state=42)),
])

#запустим кросс-валидацию
cv_scores = cross_val_score(classifier_CB, X_train, y_train, cv=16, scoring='neg_log_loss')

#обучим пайплайн на всем тренировочном датасете
classifier_CB.fit(X_train, y_train)
y_score_CB = classifier_CB.predict_proba(X_test)[:, 1]


In [11]:
classifier_RF = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

#запустим кросс-валидацию
cv_scores = cross_val_score(classifier_RF, X_train, y_train, cv=16, scoring='neg_log_loss')

#обучим пайплайн на всем тренировочном датасете
classifier_RF.fit(X_train, y_train)
y_score_RF = classifier_RF.predict_proba(X_test)[:, 1]

In [12]:
classifier_XGB = Pipeline([
    ('features',feats),
    ('classifier', xgb.XGBClassifier(random_state=42)),
])

#запустим кросс-валидацию
cv_scores = cross_val_score(classifier_XGB, X_train, y_train, cv=16, scoring='neg_log_loss')

#обучим пайплайн на всем тренировочном датасете
classifier_XGB.fit(X_train, y_train)
y_score_XGB = classifier_XGB.predict_proba(X_test)[:, 1]

Посчитаем precision/recall/f_score

In [13]:
b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score_LR)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)

roc_auc_LR = roc_auc_score(y_true=y_test, y_score=classifier_LR.predict_proba(X_test)[:,1])
log_loss_LR = log_loss(y_true=y_test, y_pred=classifier_LR.predict_proba(X_test)[:,1])
fscore_LR = fscore[ix]
precision_LR = precision[ix]
recall_LR = recall[ix]

models_results['models'].append('LogReg')
models_results['precision'].append(precision_LR)
models_results['recall'].append(recall_LR)
models_results['fscore'].append(fscore_LR)
models_results['roc_auc'].append(roc_auc_LR)
models_results['log_loss'].append(log_loss_LR)

In [14]:
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score_CB)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)

roc_auc_CB = roc_auc_score(y_true=y_test, y_score=classifier_CB.predict_proba(X_test)[:,1])
log_loss_CB = log_loss(y_true=y_test, y_pred=classifier_CB.predict_proba(X_test)[:,1])
fscore_CB = fscore[ix]
precision_CB = precision[ix]
recall_CB = recall[ix]

models_results['models'].append('CatBoost')
models_results['precision'].append(precision_CB)
models_results['recall'].append(recall_CB)
models_results['fscore'].append(fscore_CB)
models_results['roc_auc'].append(roc_auc_CB)
models_results['log_loss'].append(log_loss_CB)

In [15]:
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score_RF)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)

roc_auc_RF = roc_auc_score(y_true=y_test, y_score=classifier_RF.predict_proba(X_test)[:,1])
log_loss_RF = log_loss(y_true=y_test, y_pred=classifier_RF.predict_proba(X_test)[:,1])
fscore_RF = fscore[ix]
precision_RF = precision[ix]
recall_RF = recall[ix]

models_results['models'].append('RandomForest')
models_results['precision'].append(precision_RF)
models_results['recall'].append(recall_RF)
models_results['fscore'].append(fscore_RF)
models_results['roc_auc'].append(roc_auc_RF)
models_results['log_loss'].append(log_loss_RF)

In [16]:
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score_XGB)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)

roc_auc_XGB = roc_auc_score(y_true=y_test, y_score=classifier_XGB.predict_proba(X_test)[:,1])
log_loss_XGB = log_loss(y_true=y_test, y_pred=classifier_XGB.predict_proba(X_test)[:,1])
fscore_XGB = fscore[ix]
precision_XGB = precision[ix]
recall_XGB = recall[ix]

models_results['models'].append('XGB')
models_results['precision'].append(precision_XGB)
models_results['recall'].append(recall_XGB)
models_results['fscore'].append(fscore_XGB)
models_results['roc_auc'].append(roc_auc_XGB)
models_results['log_loss'].append(log_loss_XGB)

In [17]:
pd.DataFrame(data=models_results).sort_values('log_loss', ascending=False)

,models,precision,recall,fscore,roc_auc,log_loss
2,RandomForest,0.658873,0.792166,0.719397,0.770853,0.605110
0,LogReg,0.647184,0.838134,0.730385,0.784207,0.577721
3,XGB,0.681068,0.808180,0.739199,0.797830,0.547010
1,CatBoost,0.656969,0.845507,0.739409,0.801397,0.541362


Сделать выводы - вопрос не однозначный - Поскольку мы имеем дело с бинарной классификацией, метрикой является логарифмическая функция потерь и по этому критерию лучший результат показала модель RandomForest, но всязи с тем что задача в диагностики заболивания то необходимо максимизировать Recall а по этому критерию лучший результат показала модель CatBoost.